In [1]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Loading data

In [3]:
data=load_dataset('argilla/FinePersonas-Conversations-Email-Summaries')
data

DatasetDict({
    train: Dataset({
        features: ['conversation_id', 'email', 'maximum_brevity_summary', 'summary', 'distilabel_metadata', 'model_name'],
        num_rows: 363584
    })
})

In [4]:
data_small=data['train'].select(range(165000))
data_small

Dataset({
    features: ['conversation_id', 'email', 'maximum_brevity_summary', 'summary', 'distilabel_metadata', 'model_name'],
    num_rows: 165000
})

In [5]:
data_small=data_small.train_test_split(test_size=0.1)
data_small

DatasetDict({
    train: Dataset({
        features: ['conversation_id', 'email', 'maximum_brevity_summary', 'summary', 'distilabel_metadata', 'model_name'],
        num_rows: 148500
    })
    test: Dataset({
        features: ['conversation_id', 'email', 'maximum_brevity_summary', 'summary', 'distilabel_metadata', 'model_name'],
        num_rows: 16500
    })
})

In [6]:
data_small['train'][0]

{'conversation_id': 996,
 'email': "Subject: Re: Reaching out for expertise\n\nEmily,\n\nI had a chance to review the outline, and I think it's a great starting point. I made a few suggestions and added some terms I think would be valuable to include. Please see the attached revised outline.\n\nI've also been thinking about how we could expand our collaboration beyond the glossary. I believe there's an opportunity to create an online course that combines your expertise in science communication and my knowledge of fertility education. We could use the glossary as the foundation and build upon it with engaging content, visuals, and interactive elements.\n\nWhat do you think? I'm happy to discuss this idea further during our call on Tuesday.\n\nBest regards,\nOlivia",
 'maximum_brevity_summary': 'Olivia reviewed and revised the outline, suggesting an expansion into an online course.',
 'summary': 'Olivia reviewed the outline and made suggestions, adding valuable terms. She proposes expand

In [7]:
data_small['train'][0]['maximum_brevity_summary']

'Olivia reviewed and revised the outline, suggesting an expansion into an online course.'

In [8]:
data_small['train'][0]['summary']

'Olivia reviewed the outline and made suggestions, adding valuable terms. She proposes expanding the collaboration to create an online course that combines expertise in science communication and fertility education, using the glossary as a foundation. Olivia suggests discussing this further during the call on Tuesday.'

Dataset preprocessing

In [3]:
check_point='facebook/bart-base' #facebook/bart-large-cnn
tokenizer=AutoTokenizer.from_pretrained(check_point)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
data_input=data_small.remove_columns([
    'conversation_id', 'summary','distilabel_metadata', 'model_name'
    ])

In [11]:
data_input

DatasetDict({
    train: Dataset({
        features: ['email', 'maximum_brevity_summary'],
        num_rows: 148500
    })
    test: Dataset({
        features: ['email', 'maximum_brevity_summary'],
        num_rows: 16500
    })
})

In [12]:
max_length=1024
max_target=128
def token_func(example):
    input=tokenizer(
        example['email'],
        max_length=max_length,
        truncation=True
        )
    labels=tokenizer(
        example['maximum_brevity_summary'],
        max_length=max_target,
        truncation=True
        )
    input['labels']=labels['input_ids']
    return input

In [13]:
data_ecd=data_input.map(token_func, batched=True)

Map:   0%|          | 0/148500 [00:00<?, ? examples/s]

Map:   0%|          | 0/16500 [00:00<?, ? examples/s]

In [14]:
data_ecd

DatasetDict({
    train: Dataset({
        features: ['email', 'maximum_brevity_summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 148500
    })
    test: Dataset({
        features: ['email', 'maximum_brevity_summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 16500
    })
})

In [15]:
data_ecd=data_ecd.remove_columns(['email','maximum_brevity_summary'])
data_ecd

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 148500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16500
    })
})

Peft model
* Using LORA to fine-tune the model

In [4]:
#Loading model
model=AutoModelForSeq2SeqLM.from_pretrained(check_point)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [17]:
print(model)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [18]:
from peft import LoraConfig, get_peft_model

In [19]:
model_config=LoraConfig(
    r=8,
    task_type='SEQ_2_SEQ_LM',
    target_modules=['k_proj', 'v_proj', 'q_proj', 'out_proj']
)

In [20]:
model_peft=get_peft_model(model, model_config).to(device)
model_peft.print_trainable_parameters()

trainable params: 884,736 || all params: 140,305,152 || trainable%: 0.6306


Training

In [21]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

In [22]:
data_collator=DataCollatorForSeq2Seq(
    tokenizer, 
    model,
    #model_peft
)

In [23]:
n_epoch=10
batch_size=4
training_args=Seq2SeqTrainingArguments(
    output_dir='Summarization',
    num_train_epochs=n_epoch,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch'   
)

In [24]:
trainer=Seq2SeqTrainer(
    model,
    #model_peft,
    training_args,
    train_dataset=data_ecd['train'],
    eval_dataset=data_ecd['test'],
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

/tmp/ipykernel_149804/1622741389.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer=Seq2SeqTrainer(


In [25]:
trainer.train()

  0%|          | 0/371260 [00:00<?, ?it/s]

{'loss': 1.1177, 'grad_norm': 3.1435399055480957, 'learning_rate': 4.75e-05, 'epoch': 1.0}


  0%|          | 0/2063 [00:00<?, ?it/s]

{'eval_loss': 0.8072907328605652, 'eval_runtime': 120.1154, 'eval_samples_per_second': 137.368, 'eval_steps_per_second': 17.175, 'epoch': 1.0}
{'loss': 0.9268, 'grad_norm': 3.656320095062256, 'learning_rate': 4.5e-05, 'epoch': 2.0}


  0%|          | 0/2063 [00:00<?, ?it/s]

...

  0%|          | 0/2063 [00:00<?, ?it/s]

{'eval_loss': 0.5914831757545471, 'eval_runtime': 119.5628, 'eval_samples_per_second': 138.003, 'eval_steps_per_second': 17.255, 'epoch': 19.0}
{'loss': 0.6927, 'grad_norm': 3.764519214630127, 'learning_rate': 0.0, 'epoch': 20.0}


  0%|          | 0/2063 [00:00<?, ?it/s]

{'eval_loss': 0.591890811920166, 'eval_runtime': 119.4881, 'eval_samples_per_second': 138.089, 'eval_steps_per_second': 17.265, 'epoch': 20.0}
{'train_runtime': 54579.3888, 'train_samples_per_second': 54.416, 'train_steps_per_second': 6.802, 'train_loss': 0.7679103551082799, 'epoch': 20.0}


TrainOutput(global_step=371260, training_loss=0.7679103551082799, metrics={'train_runtime': 54579.3888, 'train_samples_per_second': 54.416, 'train_steps_per_second': 6.802, 'total_flos': 4.626931935228641e+17, 'train_loss': 0.7679103551082799, 'epoch': 20.0})

Saving model

In [ ]:
from transformers import pipeline
model.save_pretrained('./summarization_ft')

In [6]:
pipe=pipeline(
    'summarization',
    model='summarization_peft',
    #"facebook/bart-large-cnn",
    #check_point,
    #
    #max_length=1028,
    device=device)